# Mapping

In response to the 2008 U.S. Farm Bill, the U.S. Department of Agriculture's [Economic Research Service](https://www.ers.usda.gov/about-ers/) compiled a [June 2009 report to Congress](https://www.ers.usda.gov/publications/pub-details/?pubid=42729):

> According to data from the latest census (2000), about 23.5 million people, or 8.4 percent of the U.S. population, live in low-income neighborhoods that are more than a mile from a supermarket. Low-income neighborhoods are areas where more than 40 percent of the population has income less than or equal to 200 percent of the Federal poverty threshold ($44,000 per year for a family of four in 2008).

In this assessment, we'll simulate their analysis by creating geospatial maps to help us understand food access in Washington. There are three geographic region types that we'll use in this assessment:

- **Census tract** is a geographic region used in the U.S. census. It is the smallest of the three region types.
- **County** is a geographic region used for administrative purposes that can include one or more census tracts.
- **State** is a geographic region such as the State of Washington. It is the largest of the three region types.

A census tract is defined as **low access** if enough people in the tract do not have nearby access to grocery stores offering affordable and nutritious food. In urban areas, "low access" is defined as 0.5 miles; in rural areas, "low access" is defined as 10 miles.

`tl_2010_53_tract00.shp` contains the 2010 US census dataset in geospatial shapefile format only for Washington state (53). The only columns you need to use are `CTIDFP00`, the census tract identifier, and `geometry`, the geometric shape of the tract.

`food_access.csv` contains the food access dataset in tabular CSV format. Each row in the dataset corresponds to a census tract for every state in the country (not just Washington). This dataset has many columns but you only need to understand the following:

- `CensusTract` is the census tract identifier.
- `State` is the state name for the census tract.
- `County` is the county name for the census tract.
- `Urban` is a flag (0 or 1) that indicates if this census tract is an urban environment.
- `Rural` is a flag that indicates if this census tract is a rural environment.
- `LATracts_half` is a flag that indicates if this census tract is "low access" in a half mile radius.
- `LATracts10` is a flag that indicates if this census tract is "low access" in a 10 mile radius.
- `LowIncomeTracts` is a flag that indicates if this census tract is "low income".
- `POP2010` is the number of people in this census tract according to the 2010 census.
- `lapophalf` is the number of people in this census tract considered having "low access" in a half mile radius.
- `lapop10` is the number of people in this census tract considered having "low access" in a 10 mile radius.
- `lalowihalf` is similar to `lapophalf` but only counts people considered low access and low income.
- `lalowi10` is similar to `lapop10` but only counts people considered low access and low income.

In [ ]:
!pip install -q folium mapclassify

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# For testing purposes
import json
import numpy as np
from geopandas.plotting import _plot_polygon_collection
from matplotlib.collections import PatchCollection

with open("expected_idx.json") as f:
    rural_idx, rural_la_idx, urban_ha_idx, lalowi_idx = json.load(f)

class MockAxes:
    def add_collection(self, *args, **kwargs):
        pass

    def autoscale_view(self, *args, **kwargs):
        pass

def assert_patches_allclose(actual_patches, ax=MockAxes(), num_colors=None, **kwargs):
    if isinstance(kwargs.get("geoms"), str):
        kwargs["geoms"] = gpd.read_file(kwargs["geoms"]).geometry
    expected_patches = _plot_polygon_collection(ax=ax, **kwargs)
    for expected, actual in zip(expected_patches.get_paths(), actual_patches.get_paths()):
        np.testing.assert_allclose(expected.vertices, actual.vertices)
    if "color" in kwargs:
        np.testing.assert_allclose(expected_patches.get_fc(), actual_patches.get_fc())
    elif isinstance(num_colors, int):
        num_unique = len(np.unique(actual_patches.get_fc(), axis=0))
        assert num_unique == num_colors, f"expected {num_colors} colors, got {num_unique} colors"

## Task: Load in data

Write a function `load_data` that takes path for census dataset and the path for the food access dataset and returns the `GeoDataFrame` resulting from merging the two datasets on the census tract identifiers `CTIDFP00` / `CensusTract`. Assume the census tract identifier columns exist: use only these two column names. Not all census tracts have food access data.

In [ ]:
def load_data(shp_path, csv_path):
    ...


state_data = load_data("tl_2010_53_tract00.shp", "food_access.csv")
display(state_data)
assert type(state_data) == gpd.GeoDataFrame
assert list(state_data.columns) == [
    "STATEFP00", "COUNTYFP00", "TRACTCE00", "CTIDFP00", "NAME00", "NAMELSAD00", "MTFCC00",
    "FUNCSTAT00", "ALAND00", "AWATER00", "INTPTLAT00", "INTPTLON00", "geometry", "CensusTract",
    "State", "County", "Urban", "Rural", "LATracts_half", "LATracts10", "GroupQuartersFlag",
    "OHU2010", "NUMGQTRS", "PCTGQTRS", "LowIncomeTracts", "POP2010", "lapophalf", "lalowihalf",
    "lapop10", "lalowi10",
]
assert len(state_data) == 1318

## Task: Plot census tracts

Write a function `plot_census_map` that takes the merged data and returns the `Axes` that contains shapes of all the census tracts in Washington. Title the plot "Washington Census Tracts" and turn off axis labels. The output should look look like Washington state with zig-zagging census tract lines.

In [ ]:
def plot_census_map(state_data):
    ...


ax = plot_census_map(state_data)
pcs = ax.findobj(PatchCollection)
assert_patches_allclose(pcs[0], geoms=state_data.geometry)
assert len(pcs) == 1, "unexpected extra plot layers"
assert ax.get_title() == "Washington Census Tracts", "title does not match expected"
assert not ax.axison, "borders and labels must be hidden"

When given no arguments, the `dissolve` method considers the entire `GeoDataFrame` as a single group. This will be useful for plotting backgrounds later.

In [ ]:
entire_state = state_data[["geometry"]].dissolve()
entire_state

## Task: Plot census tract populations

Write a function `plot_census_population_map` that takes the merged data and return the `Axes` that plots all the census tracts in Washington where each census tract is colored according to the `POP2010` column. There will be some missing census tracts. Underneath, plot the entire state of Washington in the background color `#EEE`. Title the plot "Washington Census Tract Populations", turn off axis labels, include a legend, and increase the figure size so that the map is the same height as the legend.

In [ ]:
def plot_census_population_map(state_data):
    ...


ax = plot_census_population_map(state_data)
pcs = ax.findobj(PatchCollection)
assert_patches_allclose(pcs[0], geoms=entire_state.geometry, color="#EEE")
assert_patches_allclose(pcs[1], geoms=state_data.dropna().geometry, num_colors=183)
assert len(pcs) == 2, "unexpected extra plot layers"
assert ax.get_title() == "Washington Census Tract Populations", "title does not match expected"
assert not ax.axison, "borders and labels must be hidden"
cbar = ax.get_figure().get_axes()[-1]
assert cbar.get_label() == "<colorbar>", "missing legend"
assert ax.bbox.height == cbar.bbox.height, "map can be enlarged"

## Task: Plot county populations

Write a function `plot_county_populations_map` that takes the merged data and returns the `Axes` that plots all the counties in Washington where each county is colored according to the `POP2010` column. This will require combining all the census tract data and geometries for each county, though there will be missing data for some counties. Underneath, plot the entire state of Washington in the background color `#EEE`. Title the plot "Washington County Populations", turn off axis labels, include a legend, and increase the figure size so that the map is the same height as the legend.

In [ ]:
def plot_county_population_map(state_data):
    ...


ax = plot_county_population_map(state_data)
pcs = ax.findobj(PatchCollection)
assert_patches_allclose(pcs[0], geoms=entire_state.geometry, color="#EEE")
assert_patches_allclose(pcs[1], geoms="counties.geojson", num_colors=20)
assert len(pcs) == 2, "unexpected extra plot layers"
assert ax.get_title() == "Washington County Populations", "title does not match expected"
assert not ax.axison, "borders and labels must be hidden"
cbar = ax.get_figure().get_axes()[-1]
assert cbar.get_label() == "<colorbar>", "missing legend"
assert ax.bbox.height == cbar.bbox.height, "map can be enlarged"

## Task: Plot food access by county

Write a function `plot_food_access_by_county_map` that takes the merged data and returns a 4-tuple of `Axes` that represent the subplots in a 2-by-2 figure consisting of 4 choropleth maps:

- Top left plot titled "Low Access: Half Mile" showing the proportion of people per county who have low access to food within a half mile `lapophalf` out of the total population `POP2010`.
- Top right plot titled "Low Access + Low Income: Half Mile" showing the proportion of people per county considered low income who also have low access to food within a half mile `lalowihalf` out of the total population `POP2010`.
- Bottom left plot titled "Low Access: 10 Miles" showing the proportion of people per county who have low access to food within 10 miles `lapop10` out of the total population `POP2010`.
- Bottom right plot titled "Low Access + Low Income: 10 Miles" showing the proportion of people per county considered low income who also have low access to food within 10 miles `lalowi10` out of the total population `POP2010`.

When calling `plot`, specify the keyword arguments `vmin=0` and `vmax=1` so that the subplots all share the same scale. Underneath, plot the entire state of Washington in the background color `#EEE`. We recommend preparing subplots with `figsize=(15, 10)`. Turn off axis labels on each subplot.

In [ ]:
def plot_food_access_by_county_map(state_data):
    ...


axs = plot_food_access_by_county_map(state_data)
expected_titles = ["Low Access: Half Mile", "Low Access + Low Income: Half Mile",
                   "Low Access: 10 Miles", "Low Access + Low Income: 10 Miles"]
for ax, expected_num_colors, expected_title in zip(axs, [31, 23, 19, 16], expected_titles):
    pcs = ax.findobj(PatchCollection)
    assert_patches_allclose(pcs[0], geoms=entire_state.geometry, color="#EEE")
    assert_patches_allclose(pcs[1], geoms="counties.geojson", num_colors=expected_num_colors)
    assert len(pcs) == 2, "unexpected extra plot layers"
    assert ax.get_title() == expected_title, f"title {ax.get_title()} does not match expected"
    assert not ax.axison, "borders and labels must be hidden"

## Writeup: Food access by county

Setting aside the lack of a legend in `plot_food_access_by_county_map`, is it an effective visualization? Why or why not?

*Replace this text with your answer.*

## Task: Plot low access census tracts

Write a function `plot_census_low_access_map` that takes the merged data and returns the `Axes` that plots all census tracts (not counties) considered low access using 5 `plot` layers for the following definitions for "low access" in urban and rural tracts. For this task, do not use the `LATracts_half` or `LATracts10` columns in the merged data; the procedure described below intentionally results in different values.

1. Plot the map of Washington in the background with color `#EEE`.

1. Plot all the `Urban` and `Rural` census tracts for which we have food access data in the color `#AAA`.

1. Plot all the `Urban` and `Rural` census tracts considered low access in the default (blue) color.

   **Low access in an urban census tract** is defined by a `lapophalf` value that is at least 500 people or at least 33% of the census tract population.

   **Low access in a rural census tract** is defined by a `lapop10` value that is at least 500 people or at least 33% of the census tract population.

Finally, title the plot "Low Access Census Tracts" and turn off axis labels.

In [ ]:
def plot_census_low_access_map(state_data):
    ...


ax = plot_census_low_access_map(state_data)
pcs = ax.findobj(PatchCollection)
assert_patches_allclose(pcs[0], geoms=entire_state.geometry, color="#EEE")
urban_idx = state_data["Urban"].notna() & ~state_data.index.isin(rural_idx)
urban_la_idx = urban_idx & ~state_data.index.isin(urban_ha_idx)
error = None
for i, j, k, l in [
    [1, 2, 3, 4], # urban, rural, urban low access, rural low access
    [2, 1, 3, 4], # rural, urban, urban low access, rural low access
    [1, 2, 4, 3], # urban, rural, rural low access, urban low access
    [2, 1, 4, 3], # rural, urban, rural low access, urban low access
    [1, 3, 2, 4], # urban, urban low access, rural, rural low access
    [3, 1, 4, 2], # rural, rural low access, urban, urban low access
]:
    try:
        assert_patches_allclose(pcs[i], geoms=state_data.loc[urban_idx, "geometry"], color="#AAA")
        assert_patches_allclose(pcs[j], geoms=state_data.loc[rural_idx, "geometry"], color="#AAA")
        assert_patches_allclose(pcs[k], geoms=state_data.loc[urban_la_idx, "geometry"])
        assert_patches_allclose(pcs[l], geoms=state_data.loc[rural_la_idx, "geometry"])
        break
    except AssertionError as e:
        if error is None: # Store only the first error encountered during testing
            error = e
else: # Only raise an error if none of the possible ways to layer the plot worked
    raise error
assert len(pcs) == 5, "unexpected extra plot layers"
assert ax.get_title() == "Low Access Census Tracts", "title does not match expected"
assert not ax.axison, "borders and labels must be hidden"

## Writeup: Data-driven decision-making

What is one way that government or food access organizations could use `plot_food_access_by_county` or `plot_low_access_tracts` to shape their decisions or policies? Then, explain one limitation or concern with using the plots in the way you suggested.

*Replace this text with your answer.*

## Task: Interactive map

Although the initial report to Congress was completed in June 2009, the Economic Research Service has since then maintained an interactive map for their [**Food Access Research Atlas**](https://gisportal.ers.usda.gov/ers/GIS_FARA/lowincome_lowaccess.html). Open this interactive map, turn off the default layer "LI and LA and 1 and 10 miles", and turn on the layer "LI and LA at 1/2 and 10 miles". This layer displays:

> Low-income census tracts where a significant number or share of residents is more than 1/2 mile (urban) or 10 miles (rural) from the nearest supermarket.

Write a function `interactive_map` that returns a Folium `Map` of low income and low access census tracts in Washington. Include only `LowIncomeTracts` that are either low access at half a mile `LATracts_half` or low access at 10 miles `LATracts10` depending on whether the census tract is `Urban` or `Rural`, respectively. This dataset does not match the Food Access Research Atlas, so some differences are to be expected. It is also OK if your interactive map does not appear in the PDF printout as PDF files cannot embed interactive maps.

In [ ]:
def interactive_map(state_data):
    ...


map = interactive_map(state_data)
display(map)
geojson = next(reversed(map._children.values())).data
assert [int(d["id"]) for d in geojson["features"]] == lalowi_idx, "wrong selection"

## Writeup: Build a new supermarket

Using the interactive map above, locate the low-income low-access census tract closest to your favorite place in Washington. Then, identify a location (a specific street intersection, such as "University Way NE & NE 45th St") to add a new supermarket that would serve the people living in that census tract. Finally, explain the considerations that factored into your choice of location.

This dataset is outdated, so assume there are no supermarkets in any low-income low-access census tract even if supermarkets are present today.

*Replace this text with your answer.*